[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Graf_et_al_2019/Table_1.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Graf_et_al_2019/Table_1.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Graf_et_al_2019/Table_1.ipynb)

### reproducing Table 1 from Graf et al. 2019: https://doi.org/10.5194/acp-19-747-2019

Calculated difference in isotopic composition for vapour vs. liquid/solid in equilibrium   
Two different isotopic compositions   
Exemplifies computation of: δ18O, δ2H, excess-d 

In [3]:
from PySDM import Formulae
from PySDM.physics import si, in_unit
from IPython.display import display, HTML

In [6]:
formulae = Formulae(
    isotope_equilibrium_fractionation_factors='Majoube1970+Majoube1971+MerlivatAndNief1967',
    isotope_meteoric_water_line_excess='Dansgaard1964'
)
const = formulae.constants

alphas = {
    '18O_l': formulae.isotope_equilibrium_fractionation_factors.alpha_l_18O,
    '2H_l': formulae.isotope_equilibrium_fractionation_factors.alpha_l_2H,
    '18O_s': formulae.isotope_equilibrium_fractionation_factors.alpha_i_18O,
    '2H_s': formulae.isotope_equilibrium_fractionation_factors.alpha_i_2H
}
excess_d = formulae.isotope_meteoric_water_line_excess.excess_d

CASES = {
    'A': {'18O': -10 * const.PER_MILLE, '2H': -80 * const.PER_MILLE},
    'B': {'18O': -25 * const.PER_MILLE, '2H': -200 * const.PER_MILLE}
}
for case in CASES.values():
    case['excess'] = excess_d(delta_2H=case['2H'], delta_18O=case['18O'])

In [ ]:
# see text just below eq. (4) in the paper
alpha_20C_l = alphas['18O_l'](20 * si.K + const.T0)
alpha_20C_2H = alphas['2H_l'](20 * si.K + const.T0)
assert f"{alpha_20C_l:.4f}" == "1.0098"
assert f"{alpha_20C_2H:.4f}" == "1.0850"

In [7]:
table_data = {}
for TC in (20, 0):
    table_data[TC] = {}
    for phase in (('s', 'l') if TC==0 else ('l',)):
        table_data[TC][f'{phase}_v'] = {}
        for case, delta_v in CASES.items():
            case_data = table_data[TC][f'{phase}_v'][case] = {}
            for isotope in ('18O', '2H'):
                reference_ratio = getattr(const, f"VSMOW_R_{isotope}")
                Rv = formulae.trivia.isotopic_delta_2_ratio(
                    delta=delta_v[isotope],
                    reference_ratio=reference_ratio
                )
                R_phase = alphas[f'{isotope}_{phase}'](TC * si.K + const.T0) * Rv
                delta_phase = formulae.trivia.isotopic_ratio_2_delta(
                    ratio=R_phase,
                    reference_ratio=reference_ratio
                )
                
                case_data[f'delta_{phase}_{isotope}'] = delta_phase
                case_data[f'diff_delta_{isotope}'] = delta_phase - delta_v[isotope]
            excess_vapour = excess_d(delta_2H=delta_v['2H'], delta_18O=delta_v['18O'])
            excess_phase = excess_d(delta_2H=case_data[f'delta_{phase}_2H'], delta_18O=case_data[f'delta_{phase}_18O'])
            case_data["diff_d_excess"] = excess_phase - excess_vapour

In [8]:
table_html = "<table>"
table_html += """
  <tr style="border-bottom:1px black solid">
    <th colspan=2>composition of vapour</th>
    <th>Δ<sub>l-v</sub>@20<sup>∘</sup>C</th>
    <th>Δ<sub>l-v</sub>@0<sup>∘</sup>C</th>
    <th>Δ<sub>s-v</sub>@0<sup>∘</sup>C</th>
  </tr>
"""
row_labels = {
    'diff_delta_2H': 'δ<sup>2</sup>H',
    'diff_delta_18O': 'δ<sup>18</sup>O',
    'diff_d_excess': 'd',
}
for case in ('A', 'B'):
    for row, var in enumerate(('diff_delta_2H', 'diff_delta_18O', 'diff_d_excess')):
        table_html += f"""
          <tr{"" if row > 0 else ' style="border-top:1px black solid"'}>
            <td>{row_labels[var]}</td>
            <td>{in_unit(CASES[case][var[var.rindex("_")+1:]], const.PER_MILLE)}‰</td>
            <td>{in_unit(table_data[20]['l_v'][case][var], const.PER_MILLE):.1f}‰</td>
            <td>{in_unit(table_data[ 0]['l_v'][case][var], const.PER_MILLE):.1f}‰</td>
            <td>{in_unit(table_data[ 0]['s_v'][case][var], const.PER_MILLE):.1f}‰</td>
          </tr>
        """
table_html += "</table>"

In [9]:
display(HTML(table_html))